In [1]:
## This file implements neural networks and random forest on pyopresabsSTCC_qual.
## Due to the imbalanced data and limited cases for class 2, we implement the over-sampling method.
## For fully-connected neural networks, the accuracy is 97.46% for over-sampling data, and 95.94% after regularization.
## For random forest, the accuracy is 98.48% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 96.05% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/pyopresabsSTCC_qual.csv')
df.shape

(255, 541)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    1
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

0    218
1     33
2      4
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,ST,CC,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 540)

In [9]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,ST,CC,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 539) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218), (2, 218)]


Using TensorFlow backend.


In [12]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [13]:
############# Fully-Connected Neural Network ################

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [15]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [16]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 277us/step - loss: 1.3028 - accuracy: 0.5427 - val_loss: 0.7985 - val_accuracy: 0.6294
Epoch 2/100
457/457 [==============================] - 0s 86us/step - loss: 0.6263 - accuracy: 0.7287 - val_loss: 0.5474 - val_accuracy: 0.7868
Epoch 3/100
457/457 [==============================] - 0s 82us/step - loss: 0.4576 - accuracy: 0.8490 - val_loss: 0.4417 - val_accuracy: 0.8376
Epoch 4/100
457/457 [==============================] - 0s 79us/step - loss: 0.3558 - accuracy: 0.9059 - val_loss: 0.3580 - val_accuracy: 0.8680
Epoch 5/100
457/457 [==============================] - 0s 86us/step - loss: 0.2983 - accuracy: 0.9103 - val_loss: 0.3353 - val_accuracy: 0.8832
Epoch 6/100
457/457 [==============================] - 0s 80us/step - loss: 0.2570 - accuracy: 0.9409 - val_loss: 0.2776 - val_accuracy: 0.8883
Epoch 7/100
457/457 [==============================] - 0s 83us/step - loss: 0.2275 - accu

457/457 [==============================] - 0s 110us/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.0682 - val_accuracy: 0.9746
Epoch 58/100
457/457 [==============================] - 0s 97us/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.0728 - val_accuracy: 0.9746
Epoch 59/100
457/457 [==============================] - 0s 89us/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.9746
Epoch 60/100
457/457 [==============================] - 0s 81us/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9746
Epoch 61/100
457/457 [==============================] - 0s 83us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.9746
Epoch 62/100
457/457 [==============================] - 0s 95us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0710 - val_accuracy: 0.9746
Epoch 63/100
457/457 [==============================] - 0s 80us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.0701 - val_accuracy: 0.97

In [18]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

197/197 [==============================] - 0s 63us/step
over-sampling test accuracy: 97.46%


In [23]:
#### add dropout and regularizor
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [24]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 386us/step - loss: 10.6448 - accuracy: 0.3523 - val_loss: 6.2310 - val_accuracy: 0.4365
Epoch 2/100
457/457 [==============================] - 0s 84us/step - loss: 8.6106 - accuracy: 0.3917 - val_loss: 4.9359 - val_accuracy: 0.5025
Epoch 3/100
457/457 [==============================] - 0s 85us/step - loss: 7.3711 - accuracy: 0.4398 - val_loss: 3.7704 - val_accuracy: 0.6345
Epoch 4/100
457/457 [==============================] - 0s 83us/step - loss: 5.5649 - accuracy: 0.5098 - val_loss: 1.6053 - val_accuracy: 0.6701
Epoch 5/100
457/457 [==============================] - 0s 88us/step - loss: 4.5802 - accuracy: 0.5886 - val_loss: 1.3856 - val_accuracy: 0.6904
Epoch 6/100
457/457 [==============================] - 0s 85us/step - loss: 3.5466 - accuracy: 0.6324 - val_loss: 1.0295 - val_accuracy: 0.7259
Epoch 7/100
457/457 [==============================] - 0s 90us/step - loss: 3.5794 - acc

457/457 [==============================] - 0s 92us/step - loss: 1.3169 - accuracy: 0.7877 - val_loss: 0.6588 - val_accuracy: 0.9391
Epoch 58/100
457/457 [==============================] - 0s 84us/step - loss: 1.2860 - accuracy: 0.7746 - val_loss: 0.6802 - val_accuracy: 0.9391
Epoch 59/100
457/457 [==============================] - 0s 76us/step - loss: 1.3433 - accuracy: 0.7899 - val_loss: 0.6756 - val_accuracy: 0.9340
Epoch 60/100
457/457 [==============================] - 0s 72us/step - loss: 1.4103 - accuracy: 0.7921 - val_loss: 0.6583 - val_accuracy: 0.9188
Epoch 61/100
457/457 [==============================] - 0s 82us/step - loss: 1.5153 - accuracy: 0.7637 - val_loss: 0.6656 - val_accuracy: 0.9391
Epoch 62/100
457/457 [==============================] - 0s 74us/step - loss: 1.3045 - accuracy: 0.7702 - val_loss: 0.7452 - val_accuracy: 0.9137
Epoch 63/100
457/457 [==============================] - 0s 71us/step - loss: 1.3440 - accuracy: 0.7637 - val_loss: 0.6652 - val_accuracy: 0.939

In [26]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

197/197 [==============================] - 0s 75us/step
over-sampling test accuracy: 95.94%


In [27]:
############## Random Forest ##############

In [28]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.48%


In [30]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.95698925 0.97849462 0.96703297 0.94444444 0.95555556]
0.9605033676001419
